In [1]:
from pygraphblas import *
from itertools import repeat
import random
random.seed(11)

In [2]:
M = Matrix.from_lists(
    [0, 0, 0, 0,
    1, 1, 1, 1,
    2, 2, 2, 2,
    3, 3, 3, 3,
    4, 4, 4, 4,
    5, 5, 5, 5, 5,
    6, 6, 6,
    7, 7, 7, 7],
    [0, 2, 3, 6,
    1, 2, 3, 7,
    0, 2, 4, 6,
    0, 1, 3, 5,
    0, 2, 4, 6,
    1, 3, 5, 6, 7,
    0, 4, 6,
    1, 3, 5, 7],
    list(repeat(1.0, 32)))

@uop(FP64)  
def random_scaler(x):  
    return random.random() * x + 0.0001


typedef void (*_random_scaler_uop_function)(double*, double*);

(float64*, float64*) -> none


In [3]:
def louvain_cluster(graph, max_iters=2):
    if not graph.square:
        raise lib.DimensionMismatch('First input matrix must be square.')
    rows = graph.nrows
    
    ApAT = graph.transpose()
    k = graph.reduce_vector()
    m = k.reduce_int() / 2.0   
    print(m, k)
    
    S = Matrix.identity(BOOL, rows)
    S_row = Vector.from_type(BOOL, rows)
    mask = Vector.dense(BOOL, rows)
    
    vertices_changed = True
    iters = 0
    while vertices_changed and iters < max_iters:
        vertices_changed = False
        for i in range(rows):
            if i in k:
                print('===Start of vertex: ', i)
                S.extract_vector(i, out=S_row, desc=descriptor.tooo)
                Mask = Matrix.from_type(BOOL, rows, rows)
                Mask[i,:] = mask
                S.apply(lib.GrB_IDENTITY_BOOL, out=S, mask=Mask, desc=descriptor.oocr)
                v = ApAT[:,i]
                w = v.dup()
                w.assign_scalar(-k[i]/m, accum=times)
                v += w
                q = v @ S
                print(q.to_lists(), "modularity changed")
                kappa = q.reduce_float(monoid=lib.GxB_MAX_FP64_MONOID)
                t = (q == kappa)
                t = t.nonzero()
                while False: # len(t) != 1:
                    print('breaking ties')
                    p = t.apply(random_scaler)
                    max_p = p.reduce_float(monoid=lib.GxB_MAX_FP64_MONOID)
                    t = (q == max_p)
                    t = t.nonzero()
                S[i,:] = q[t]
                if t != S_row:
                    vertices_changed = True
                    print('new S')
                else:
                    print('No change.')
                print('end of vertex', i)
        iters += 1
    return S

In [4]:
ans = louvain_cluster(M)

16.0 <Vector (8: 8)>
===Start of vertex:  0
[[1, 2, 3, 4, 5, 6, 7], [-0.25, 0.75, 0.75, -0.25, -0.25, 0.75, -0.25]] modularity changed
new S
end of vertex 0
===Start of vertex:  1
[[2, 3, 4, 5, 6, 7], [0.5, 0.5, -0.25, -0.25, -0.5, 0.75]] modularity changed
new S
end of vertex 1
===Start of vertex:  2
[[2, 3, 4, 5, 6, 7], [0.75, 0.5, 0.75, -0.25, 1.5, -0.5]] modularity changed
new S
end of vertex 2
===Start of vertex:  3
[[2, 3, 4, 5, 6, 7], [0.75, 0.75, -0.25, 0.75, 0.25, 0.5]] modularity changed
new S
end of vertex 3
===Start of vertex:  4
[[2, 3, 5, 6, 7], [0.5, 0.5, -0.5, 2.25, -0.5]] modularity changed
new S
end of vertex 4
===Start of vertex:  5
[[2, 3, 5, 6, 7], [0.375, 0.375, 0.6875, -0.25, 1.375]] modularity changed
new S
end of vertex 5
===Start of vertex:  6
[[2, 3, 5, 6, 7], [0.625, 0.625, -0.1875, 1.4375, -0.5625]] modularity changed
new S
end of vertex 6
===Start of vertex:  7
[[2, 3, 5, 6, 7], [0.5, 0.5, 0.75, -1.0, 1.5]] modularity changed
new S
end of vertex 7
===Start

In [5]:
ans.to_lists()

[[0, 1, 1, 1, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 7, 7],
 [6, 2, 3, 5, 6, 2, 3, 5, 6, 2, 3, 5, 6, 2, 3, 5],
 [True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True]]